In [1]:
# 十大事件
event_list = ['trafficSource', 'userRegisteration', 'mainPageView', 'productPageView', 'categoryPageView',
              'activityPageView', 'search', 'addToCart', 'checkout', 'purchase']

bh_list = ['traffic', 'viewproduct', 'add', 'viewcategory', 'viewmainpage', 'register', 'checkout', 'search',
           'purchase', 'viewactivity']

In [2]:
import pandas as pd
import numpy as np 

In [3]:
from datetime import date, timedelta
start_date = date(2018, 6, 26)
end_date = date(2021, 4, 30)
delta = timedelta(days=1)

durations = []

while start_date <= end_date:
    date = start_date.strftime("%Y-%m-%d")
    durations.append(date)
    start_date += delta

In [4]:
len(durations)

1040

# 統整行為資料
#### 合併資料

In [5]:
# 合併檔案
# ht: 時間戳記(用 Unix Timestamp, 用 pd.to_datetime 即可轉換成日期格式(UTC)，轉成台灣時區要再加 8小時)
# did: 裝置識別碼
# dc: 瀏覽裝置
def merge_data(events, dates):
    i = 1
    for event in events:
        for date_str in dates:
            path = './dataset/91APP_BehaviorData/' + event + '_' + date_str + '.csv'

            if i == 1:
                df = pd.read_csv(path, encoding = "utf-8", engine='python')
            else:
                df1 = pd.read_csv(path, encoding = "utf-8", engine='python')
                df = pd.concat([df, df1])
            i += 1
    df['DateTime'] = pd.to_datetime(df.ht, unit='ms')

    df = df.sort_values(by = ['did', 'dc', 'DateTime'])
    
    return df

#### 處理資料

In [6]:
# df: 要拆分的 dataframe
# cs_list: df 中有的 cs(no nan)
def split_df(df, cs_list):
    df.reset_index(inplace=True)
    cs_size = len(cs_list)
    splits = [0]
    
    for i in range(1, cs_size):
        cs = cs_list[i]
        index = df.index[df.cs == cs][0]
        splits.append(index)
    
    df_list = []
    for j in range(cs_size):
        if j == cs_size -1:
            start = splits[j]
            result_df = df.iloc[start:, :]
        else:
            start = splits[j]
            end = splits[j + 1]
            result_df = df.iloc[start: end, :]

        df_list.append(result_df)
    
    return df_list

In [7]:
# df: 切完後，每個 session 為一個 dataframe
def integrate(df, did, dc, cs, date):
    uid_list = df.uid.unique()
    uid_list = uid_list[~pd.isna(uid_list)]

    if len(uid_list) == 0:
        uid = ''
    else:
        uid = uid_list[0]

    data = {
        'did': did,
        'dc': dc,
        'uid': uid,
        'cs': cs,
        'date': date
    }

    for bh in bh_list:
        data[bh] = len(df[df.bh == bh])

    return data

In [8]:
# merged_data: 行為資料依照所要的事件及日期區間合併後的 data
def process_data(merged_data, date):
    non_cs = []
    multi_cs = []
    result = []

    did_list = merged_data.did.unique()
    print('did length:', len(did_list))

    for did in did_list:
        did_df = merged_data[merged_data.did == did][['dc', 'did', 'uid', 'DateTime', 'bh', 'cs', 'pr', 'pr1id']]
        dc_list = did_df.dc.unique()
        dc_list = dc_list[~pd.isna(dc_list)]

        for dc in dc_list:
            cs_list = did_df.cs.unique()
            cs_list = cs_list[~pd.isna(cs_list)]

            # 沒有廣告來源，就不進行統計
            if len(cs_list) == 0:
                non_cs.append(did)
                continue

            if len(cs_list) == 1:
                cs = cs_list[0]
                session_df = did_df
                result.append(integrate(session_df, did, dc, cs, date))

            if len(cs_list) > 1:
                multi_cs.append(did)
                session_df_list = split_df(did_df, cs_list)

                for session_df in session_df_list:
                    df_cs = session_df.cs.unique()
                    df_cs = df_cs[~pd.isna(df_cs)]
                    result.append(integrate(session_df, did, dc, df_cs[0], date))

    return non_cs, multi_cs, result

#### 輸出時間區段內整理後資料

In [9]:
def export_duration_data(start_date, end_date):
    start = str(start_date)
    end = str(end_date)
    file_name = start + ' to ' + end
    delta = timedelta(days=1)
    i = 1

    while start_date <= end_date:
        print (start_date.strftime("%Y-%m-%d"))
        date = start_date.strftime("%Y-%m-%d")
        date_list = [str(date)]
        
        daily_data = merge_data(event_list, date_list)
        daily_data = daily_data[['dc', 'did', 'uid', 'DateTime', 'bh', 'cs', 'cm', 'cn', 'pr', 'pr1id']]
        
        if i == 1:
            duration_data = daily_data
        else:
            duration_data = pd.concat([duration_data, daily_data])

        non_cs, multi_cs, result = process_data(daily_data, date)

        if i == 1:
            result_pd = pd.DataFrame(result)
        else:
            result_pd1 = pd.DataFrame(result)
            result_pd = pd.concat([result_pd, result_pd1])

        i += 1
        start_date += delta

    duration_data.to_csv(file_name + '_origin.csv', index = False, encoding = "utf-8")
    result_pd.to_csv(file_name + '_process.csv', index = False)

## Start

In [10]:
from datetime import date, timedelta
start_date = date(2018, 6, 26)
end_date = date(2018, 7, 1)
export_duration_data(start_date, end_date)

2018-06-26
did length: 6509
2018-06-27
did length: 9652
2018-06-28
did length: 7726
2018-06-29
did length: 6523
2018-06-30
did length: 8286
2018-07-01
did length: 8497


# 確認所有型位資料中有的 uid

In [42]:
from datetime import date, timedelta
start_date = date(2018, 6, 26)
end_date = date(2018, 12, 31)
delta = timedelta(days=1)

durations = []

while start_date <= end_date:
    date = start_date.strftime("%Y-%m-%d")
    durations.append(date)
    start_date += delta

In [43]:
len(durations)

189

In [44]:
def get_bh_uids(events, dates):
    uids = []
    i = 1
    
    for date_str in dates:
        for event in events:
            print(i)
            i += 1
            path = './dataset/91APP_BehaviorData/' + event + '_' + date_str + '.csv'

            df = pd.read_csv(path, encoding = "utf-8", engine='python')
            
            uid_list = df.uid.unique()
            uid_list = list(uid_list[~pd.isna(uid_list)])
            
            uids += uid_list
            uids = list(set(uids)) # remove duplicate
  
    return uids

In [45]:
behavior_uids = get_bh_uids(event_list, durations)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890


In [47]:
print('2018.06.26~2018-12-31 行為資料 uid 數量:', len(behavior_uids))

2018.06.26~2018-12-31 行為資料 uid 數量: 77579


# 統計有 purchase 的部分有幾個人

In [54]:
purchased_data = merge_data(['purchase'], durations)

In [55]:
len(purchased_data)

303823

In [56]:
purchased_data

,mid,dc,did,uid,ht,ul,geoid,bh,tr,ti,pr1id,pr1pr,pr1qt,cu,DateTime
116,123,app,00002073-5d6e-4b6f-bd68-867d91adab82,Klr7FyHPS3MzyYxwByJYUi%2BmIa%2BRN56kB4G7pQnCd3...,1603370906143,zh-Hant-TW,TW,purchase,1780.0,TG201022W01356,5895394,1780.0,1,TWD,2020-10-22 12:48:26.143
356,123,app,00002073-5d6e-4b6f-bd68-867d91adab82,Klr7FyHPS3MzyYxwByJYUi%2BmIa%2BRN56kB4G7pQnCd3...,1607398729961,zh-Hant-TW,TW,purchase,2412.0,TG201208M01016,6682804,2412.0,1,TWD,2020-12-08 03:38:49.961
117,123,web,0000cef2-3b63-42d3-5a3a-38b6d81a3c22,4QfQ5riaQWrf6Rc1LI%2B0rZ59vy1IzD1HRNwwAORbyBk%3D,1575134249850,zh-tw,TW,purchase,1275.0,TG191201B00280,5954277,1275.0,1,TWD,2019-11-30 17:17:29.850
29,123,app,0000d574-a309-48d3-91cf-0d120c287d80,D0amEkkLijMor1dhdY45wpCEIw7zKTndg%2BguBRfvlzQ%3D,1543555896889,zh-TW,TW,purchase,1480.0,TG181130P00782,4935641,1480.0,1,TWD,2018-11-30 05:31:36.889
54,123,app,0000d574-a309-48d3-91cf-0d120c287d80,D0amEkkLijMor1dhdY45wpCEIw7zKTndg%2BguBRfvlzQ%3D,1604928429307,zh-TW,TW,purchase,1460.0,TG201109X02136,4935641,1460.0,1,TWD,2020-11-09 13:27:09.307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,123,web,ffffd6da-5b3d-4411-ccec-d45262fe5468,DRX5HuTFUJYEP2bRrrbAHRobQthqHKlNKqNslEvQt5w%3D,1570429366124,zh-tw,TW,purchase,4284.0,TG191007Q00500,5797128,2499.0,1,TWD,2019-10-07 06:22:46.124
84,123,app,NaN,ZuHP185KlzPT2jCScJg2H52mm3%2F%2BtgWzW6rCDzPOff...,1534785643591,zh-TW,TW,purchase,1880.0,TG180821B00297,4530437,1880.0,1,TWD,2018-08-20 17:20:43.591
0,123,app,NaN,NaN,1539154162896,zh-TW,TW,purchase,2029.0,TG181010Q01441,4671376,199.0,1,TWD,2018-10-10 06:49:22.896
1,123,app,NaN,NaN,1539154162896,zh-TW,TW,purchase,2029.0,TG181010Q01441,4683239,350.0,1,TWD,2018-10-10 06:49:22.896


In [58]:
len(purchased_data.did.unique())

129307

In [59]:
len(purchased_data.uid.unique())

96771

In [61]:
p_uid = purchased_data.uid.unique()

In [62]:
p_uid = p_uid[~pd.isna(p_uid)]

In [63]:
len(p_uid)

96770

In [20]:
path = './dataset/91APP_PeopleView_Profile_20210511.xlsx'
people_df = pd.read_excel(path)

In [21]:
len(people_df['91APP_uid'].unique())

9827

In [22]:
people_ids = people_df['91APP_uid'].unique()
people_ids = people_ids[~pd.isna(people_ids)]
len(people_ids)

9827

In [71]:
purchased_data.to_csv('行為資料-purchase 合併.csv', index = False)

In [78]:
peopleview_ids = []

for pid in people_ids:

    if pid in p_uid:
        peopleview_ids.append(pid)

In [79]:
len(peopleview_ids)

0